### **0. Import libraries**

In [1]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.6 MB/s eta 0:00:0000:01:00:01


In [2]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from xgboost import XGBRegressor
from scipy.special import huber
import numpy as np
import seaborn as sns
from statsmodels.tools.tools import maybe_unwrap_results
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from multiprocessing import Manager
from multiprocessing.pool import Pool
import time
RANDOM_STATE = 2023
TEST_SIZE = 0.2

In [3]:
# from google.colab import drive
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# drive.mount('/content/drive')
# file_path = '/content/drive/MyDrive/Colab Notebooks/'
file_path = './'

### **1. Load data**

In [4]:
raw = pd.read_csv(file_path+'finalish_df_4.csv').iloc[:,1:]
print('Dataframe shape: ',raw.shape)
print('Dataframe info: ',raw.info())
display(raw.head(2))

Dataframe shape:  (547651, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547651 entries, 0 to 547650
Data columns (total 31 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ZCTA                               547651 non-null  int64  
 1   year_month                         547651 non-null  int64  
 2   ins_1_no_bin_raw                   547651 non-null  float64
 3   ins_2_add_acres_raw                547651 non-null  float64
 4   ins_3_norms                        547650 non-null  float64
 5   ins_4_add_acres_norms              547650 non-null  float64
 6   ins_5_norm_bins_acres              547650 non-null  float64
 7   ins_6_bins_raw                     547651 non-null  float64
 8   zip_pm25                           541289 non-null  float64
 9   zip_elevation                      545321 non-null  float64
 10  elevation_difference               210990 non-null  float64
 11  zip_wspd

,ZCTA,year_month,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_bins_acres,ins_6_bins_raw,zip_pm25,zip_elevation,...,percent_pop_female_age_under_5,percent_pop_male_age_5to9,percent_pop_male_age_under_5,total_population,percent_bach_deg_grad_new,percent_high_school_grad_new,percent_pop_age_15to19_new,percent_pop_female_age_15to19_new,percent_pop_male_age_10to14_new,percent_pop_male_age_15to19_new
0,89010,199101,0.014313,2.171619,2.193307,0.002237,0.024999,24.261588,12.450976,4784.0,...,1.8,4.5,2.4,275.6,14.272763,56.603842,13.526364,14.514991,3.99958,8.781446
1,89010,199102,0.003401,1.473353,0.522705,0.001519,0.017402,16.875655,11.255854,4784.0,...,1.8,4.5,2.4,275.6,14.272763,56.603842,13.526364,14.514991,3.99958,8.781446


In [5]:
raw_expm25 = raw.loc[~raw['zip_pm25'].isna()].copy()
print('Dataframe info: ',raw_expm25.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541289 entries, 0 to 547650
Data columns (total 31 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ZCTA                               541289 non-null  int64  
 1   year_month                         541289 non-null  int64  
 2   ins_1_no_bin_raw                   541289 non-null  float64
 3   ins_2_add_acres_raw                541289 non-null  float64
 4   ins_3_norms                        541288 non-null  float64
 5   ins_4_add_acres_norms              541288 non-null  float64
 6   ins_5_norm_bins_acres              541288 non-null  float64
 7   ins_6_bins_raw                     541289 non-null  float64
 8   zip_pm25                           541289 non-null  float64
 9   zip_elevation                      541289 non-null  float64
 10  elevation_difference               209430 non-null  float64
 11  zip_wspd                           5412

In [6]:
raw.loc[~raw['households_count'].isna()]['ZCTA'].nunique()

1612

In [7]:
raw_stage1 = raw_expm25.loc[~raw_expm25['households_count'].isna()].copy()
print('Dataframe info: ',raw_stage1.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534029 entries, 0 to 547650
Data columns (total 31 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ZCTA                               534029 non-null  int64  
 1   year_month                         534029 non-null  int64  
 2   ins_1_no_bin_raw                   534029 non-null  float64
 3   ins_2_add_acres_raw                534029 non-null  float64
 4   ins_3_norms                        534028 non-null  float64
 5   ins_4_add_acres_norms              534028 non-null  float64
 6   ins_5_norm_bins_acres              534028 non-null  float64
 7   ins_6_bins_raw                     534029 non-null  float64
 8   zip_pm25                           534029 non-null  float64
 9   zip_elevation                      534029 non-null  float64
 10  elevation_difference               206775 non-null  float64
 11  zip_wspd                           5340

In [8]:
#TO REMOVE AFTER I GET MORE COMPUTING POWER
# raw_stage1 = raw_stage1.loc[raw_stage1.ZCTA.isin(list(raw_stage1.ZCTA.unique()[:3]))]
# raw_stage1.shape

### **2. Feature engineering**

In [9]:
#look at stats first
raw_stage1.describe()

,ZCTA,year_month,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_bins_acres,ins_6_bins_raw,zip_pm25,zip_elevation,...,percent_pop_female_age_under_5,percent_pop_male_age_5to9,percent_pop_male_age_under_5,total_population,percent_bach_deg_grad_new,percent_high_school_grad_new,percent_pop_age_15to19_new,percent_pop_female_age_15to19_new,percent_pop_male_age_10to14_new,percent_pop_male_age_15to19_new
count,534029.000000,534029.000000,534029.000000,534029.000000,534028.000000,534028.000000,534028.000000,5.340290e+05,534029.000000,534029.000000,...,534029.000000,534029.000000,534029.000000,534029.000000,534029.000000,534029.000000,534029.000000,534029.000000,534029.000000,534029.000000
mean,93654.407944,200455.402399,0.028649,62.611486,4.432021,0.064825,0.703011,6.790178e+02,10.238584,1279.628382,...,7.416529,7.125835,7.650411,20990.355004,25.066798,75.272856,7.672350,7.269897,6.955869,5.993247
std,1823.233803,807.434254,0.098733,386.907416,14.833164,0.405365,4.304165,4.108757e+03,5.976172,1724.907449,...,4.572885,3.888823,4.484966,20758.000756,16.913328,17.381111,4.059558,4.726107,4.294546,3.927292
min,89010.000000,199101.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-230.000000,...,0.000000,0.000000,0.000000,2.000000,0.010000,0.486379,0.010000,0.010000,0.010000,0.010000
25%,92258.000000,199712.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,6.011667,143.000000,...,4.900000,5.000000,5.000000,1727.100000,12.085806,68.894423,5.360227,4.874479,4.854490,4.089742
50%,93647.000000,200412.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,8.819333,503.000000,...,6.800000,6.900000,7.200000,16237.500000,20.833634,80.708626,7.331364,6.643380,6.659300,5.653644
75%,95377.000000,201112.000000,0.013628,4.036542,2.096536,0.004149,0.047761,4.644233e+01,13.103636,1622.000000,...,8.900000,8.600000,9.400000,34693.000000,34.906226,87.473210,9.302500,8.677616,8.369120,7.217545
max,97635.000000,201812.000000,22.098205,97674.455495,1719.864507,103.902397,1093.379388,1.027844e+06,88.450000,10572.000000,...,67.600000,44.700000,81.300000,109414.000000,95.000000,95.000000,68.718182,75.011415,95.000000,95.000000


In [10]:
#make year values smaller
raw_stage1['year_trend'] = raw_stage1.apply(lambda x: x['year'] - (raw_stage1['year'].min()-1),axis=1)

#minmax scaling for elevation - elastic net sensitive to variables on different scales
elevation = MinMaxScaler().fit_transform(np.array(raw_stage1['elevation_difference']).reshape(-1,1))
raw_stage1['s_elevation_difference'] = elevation.reshape(elevation.shape[0],)

# #OHE  for month
raw_stage1['month'] = raw_stage1['year_month'].astype(str).apply(lambda x: x[-2:])
mth_enc = OneHotEncoder(handle_unknown='error',drop='first',sparse_output=False) 
mth_ohc_values = mth_enc.fit_transform(np.array(raw_stage1['month']).reshape(-1,1))
mth_ohc_labels = mth_enc.get_feature_names_out(['month'])
raw_stage1 = pd.concat([raw_stage1.reset_index(drop=True), pd.DataFrame(mth_ohc_values, columns = mth_ohc_labels).reset_index(drop=True)],axis=1)
print(mth_ohc_labels)

# #OHE for ZCTA
zc_enc = OneHotEncoder(handle_unknown='error',drop='first',sparse_output=False) 
zc_ohc_values = zc_enc.fit_transform(np.array(raw_stage1['ZCTA']).reshape(-1,1))
zc_ohc_labels = zc_enc.get_feature_names_out(['ZCTA'])
raw_stage1 = pd.concat([raw_stage1.reset_index(drop=True), pd.DataFrame(zc_ohc_values, columns = zc_ohc_labels).reset_index(drop=True)],axis=1)
print(zc_ohc_labels)

KeyboardInterrupt: 

In [ ]:
raw_stage1.shape


In [ ]:
raw_stage1.head(2)

In [ ]:
raw_stage1.columns

In [ ]:
#create data
def create_stage1_data(instrument_var_name):
  #list variables
  stage_1_yvar  = 'zip_pm25'
  time_vars = ['year_trend'] + list(mth_ohc_labels)
  iv_vars =  [instrument_var_name] #['ins_1_no_bin_raw', 'ins_2_add_acres_raw','ins_3_norms', 'ins_4_add_acres_norms', 'ins_5_norm_bins_acres','ins_6_bins_raw']
  env_vars = ['s_elevation_difference']
  dem_vars = [ 
          'total_population','households_count',
          'median_income_per_household', 'percent_high_school_grad_new', 'percent_bach_deg_grad_new', 
          'percent_pop_age_under_5', 'percent_pop_age_5to9', 'percent_pop_age_10to14','percent_pop_age_15to19_new',
          # 'percent_pop_male_age_under_5', 'percent_pop_male_age_5to9', 'percent_pop_male_age_10to14_new', 'percent_pop_male_age_15to19_new'
          'percent_pop_female_age_under_5', 'percent_pop_female_age_5to9','percent_pop_female_age_10to14', 'percent_pop_female_age_15to19_new'] #PENDING

  stage_1_xvars = time_vars + iv_vars + env_vars + dem_vars + list(zc_ohc_labels)

  X = raw_stage1[stage_1_xvars]
  y = raw_stage1[stage_1_yvar]

  return X, y

In [ ]:
def train_test_panel_split(X,y,test_size=TEST_SIZE):
  #split into train vs test
  last_train_idx = round(X.shape[0]*(1-test_size))

  X_train = X.iloc[:last_train_idx] 
  X_test = X.iloc[last_train_idx:]

  y_train = y.iloc[:last_train_idx]
  y_test = y.iloc[last_train_idx:]
  
  return X_train, X_test, y_train, y_test


In [ ]:
#actual code for featuregeneration
instruments = ['ins_1_no_bin_raw', 'ins_2_add_acres_raw','ins_3_norms', 'ins_4_add_acres_norms', 'ins_5_norm_bins_acres','ins_6_bins_raw']
# X, y = create_stage1_data(instrument_var_name=instruments[4])
# X_train, X_test, y_train, y_test = train_test_panel_split(X,y)

### **3. Model selection**

In [ ]:
#Define loss functions

## ADJUST FOR CLUSTER STD ERRORS

def get_rmse(model,X,y):
    return ((np.array(y) - model.predict(X))**2).mean()    

def get_mae(model,X,y):
    return (abs(np.array(y) - model.predict(X))).mean()    

def get_huber_loss(model,X,y,delta=5):
    r = np.array(y) - model.predict(X)
    return sum(huber(delta,r))

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
#Define hyperparmeters
hyperparams = {'booster':'gbtree', 
               'objective':'reg:squarederror', 
               'tree_method':'hist', 
               'nthread':16,
               # 'n_jobs':2,
               'gamma':0,
               'eval_metric':'rmse',
               'num_parallel_tree':20,         #number of trees to be running in the model
               'max_depth' : [6,12,18],         #max tree depth
               'subsample':[0.2,0.4,0.6],       #proportion of training instances subsampled
               'reg_lambda':[0.01, 0.1, 1],     #L2 reg
               'learning_rate': [0.01,0.03,0.05],
               'random_state' :RANDOM_STATE
              }
       #,
keys = list(hyperparams.keys())
hyperparams_df = pd.DataFrame(np.array(np.meshgrid(*[hyperparams[key_i] for key_i in keys])).T.reshape(-1, len(keys)))
hyperparams_df.columns = keys 
hyperparams_df.head()

In [ ]:
def select_models(hyperparams_df,X_train,y_train):
    results = {'hyperparams':[], 'rmse_train': [], 'rmse_test': []}

    folds = TimeSeriesSplit(n_splits=5) #default value

    #implement grid search
    for i in tqdm(range(hyperparams_df.shape[0])): #to use fullhyperparamset for actual run
        #get hyperparams
        hyperparams = dict(hyperparams_df.iloc[i])

        #initialize empty lists
        rmse_trains =[]; rmse_tests =[]; 
        # huber_trains=[]; huber_tests=[]   #we should care more about squared errors because "outliers" arising from wildfires are a big part of what we are trying to predict.

        for fold_count,fold in enumerate(folds.split(X_train)):
            print(fold_count)
            #get data
            X_cvtrain = X_train.loc[fold[0]]  ; X_val = X_train.loc[fold[1]]
            y_cvtrain = y_train.loc[fold[0]]  ; y_val = y_train.loc[fold[1]]
            
            # print(X_cvtrain.shape)

            #fit model
            tmp_model = XGBRegressor(**hyperparams).fit(X_cvtrain,y_cvtrain,verbose=0)  
            # print(get_rmse(tmp_model, X_cvtrain, y_cvtrain))
            #append results
            rmse_trains.append(get_rmse(tmp_model, X_cvtrain, y_cvtrain))
            rmse_tests.append(get_rmse(tmp_model, X_val, y_val))
            # huber_trains.append(get_huber_loss(tmp_model, X_cvtrain, y_cvtrain))  
            # huber_tests.append(get_huber_loss(tmp_model, X_val  , y_val))

        results['hyperparams'].append(hyperparams)
        results['rmse_train'].append(np.mean(rmse_trains))
        results['rmse_test'].append(np.mean(rmse_tests))
        # results['huber_loss_train'].append(np.mean(huber_trains))
        # results['huber_loss_test'].append(np.mean(huber_tests))
    
    best_hyperparams = pd.DataFrame(results).sort_values('rmse_test',ascending=True).iloc[0]['hyperparams']   
    return best_hyperparams, pd.DataFrame(results).sort_values('rmse_test',ascending=True)


In [ ]:
#create models for each instrument and save model output

instruments = ['ins_3_norms', 'ins_4_add_acres_norms']
models = dict()

train_rmses = [] ; test_rmses = []
train_maes = []  ; test_maes  = []

for i in tqdm(range(len(instruments))):
  #load data with correct instrument name
  print ('starting for instrument {}:'.format(instruments[i]))
  X, y = create_stage1_data(instrument_var_name=instruments[i])
  X_train, X_test, y_train, y_test = train_test_panel_split(X,y)
  X_train = X_train.iloc[:,:50]

  #select and train best performing cross validated model
  # best_hyperparams, results = select_models(hyperparams_df,X_train,y_train)  #can get params for each model using get_xgb_params()
  best_hyperparams, results = select_models(hyperparams_df,X_train,y_train) 
  print(best_hyperparams)
  models['stage1_xgb_model_'+str(i+1)] = XGBRegressor(**best_hyperparams).fit(X_train,y_train,verbose=0)

  #save model
  filename = file_path+'stage1_xgb_model_'+str(i+1) + '.json'
  models['stage1_xgb_model_'+str(i+1)].save_model(filename) 

  #save train and test rmses / maes
  train_rmses.append(get_rmse(models['stage1_xgb_model_'+str(i+1)],X_train,y_train))
  test_rmses.append( get_rmse(models['stage1_xgb_model_'+str(i+1)],X_test,y_test) )

  train_maes.append( get_mae(models['stage1_xgb_model_'+str(i+1)],X_train,y_train) )
  test_maes.append( get_mae(models['stage1_xgb_model_'+str(i+1)],X_test,y_test))

all_results = pd.DataFrame(
    {
      'model_name': ['stage1_xgb_model_'+str(i+1) for i in range(len(instruments))],
      'train_rmse': train_rmses, 'test_rmse': test_rmses, 
      'train_mae': train_maes, 'test_mae': test_maes
    }
)

all_results.to_csv(file_path+'stage1_xgb_model_results.csv')

In [ ]:
#analyze best hyperparams for each model config
best_hyperparams_all_models = [model.get_xgb_params() for model in models.values()]

for idx, name in enumerate(models.keys()):
  best_hyperparams_all_models[idx]['model_name'] = name 

best_hyperparams_all_models_df = pd.DataFrame(best_hyperparams_all_models)

final_results = pd.merge(best_hyperparams_all_models_df[['model_name']+list(hyperparams.keys())],
                         all_results ,
                         how='left', 
                         on='model_name')

display(final_results)

### **4. Model evaluation**

#### 4a. RMSEs

In [ ]:
#see above. 

#### 4b. MAEs

In [ ]:
#see above.

#### 4c. Adj. R-squared

In [ ]:
#not applicable for non-linear models.

### **5. Feature importances / inference**

In [ ]:
#code for loading xgb model object
instrument_choice = 5

best_xgb_model = XGBRegressor()
best_xgb_model.load_model(file_path+'stage1_xgb_model_{}.json'.format(str(instrument_choice)))

#### 5a. Built-in

In [ ]:
feature_category_dict = {'year_trend':'year', 
                         's_elevation_difference':'elevation'}  #check if need to remove elevation

for i in list(mth_ohc_labels): 
  feature_category_dict[i]='month'

for i in list(zc_ohc_labels): 
  feature_category_dict[i]='geography'

for i in ['ins_1_no_bin_raw', 'ins_2_add_acres_raw','ins_3_norms', 'ins_4_add_acres_norms', 'ins_5_norm_bins_acres','ins_6_bins_raw']:
  feature_category_dict[i]='instrument'

for i in ['total_population','households_count','percent_pop_age_under_5', 'percent_pop_age_5to9', 'percent_pop_age_10to14','percent_pop_age_15to19_new']:
  feature_category_dict[i]='population'

for i in ['mean_income_per_household','median_income_per_household','percent_high_school_grad_new','percent_bach_deg_grad_new']:
  feature_category_dict[i]='social.status'

for i in ['percent_pop_female_age_under_5', 'percent_pop_female_age_5to9','percent_pop_female_age_10to14', 'percent_pop_female_age_15to19_new']:
  feature_category_dict[i]='gender'
  

In [ ]:
importances = pd.DataFrame({'variable_name':best_xgb_model.get_booster().feature_names, 'importance':best_xgb_model.feature_importances_})
importances['category'] = importances['variable_name'].apply(lambda x: feature_category_dict[x])
importances[['category','importance']].boxplot(by='category')

In [ ]:
importances[['category','importance']].groupby('category').mean().sort_values(by='importance').plot(kind='barh')

In [ ]:
print('Feature importances: ')
feat_importances = pd.Series(best_xgb_model.feature_importances_, 
                             index=best_xgb_model.get_booster().feature_names)
feat_importances.nlargest(10).sort_values().plot(kind='barh')

In [ ]:
#feature importance heat map by ZCTA
geos = importances.loc[importances['category']=='geography']
geos['ZCTA'] = geos['variable_name'].apply(lambda x: x.split('_')[1]).astype(int)

#find a way to plot heat map

#### 5b. Permutation importance

#### 5c. SHAP